In [1]:
# | default_exp gsc_client


In [1]:
# | export

import pickle
from datetime import datetime, timedelta
from pathlib import Path
from typing import Optional, Tuple, List, Dict

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request as GoogleRequest


In [2]:
# | export
class GSCAuth:
    """Google Search Console Authentication Handler"""

    def __init__(
        self,
        secrets_file: str = "./client_secrets.json",
        token_file: str = "./token.pickle",
    ):
        self.secrets_file = secrets_file
        self.token_file = Path(token_file)
        self.credentials = self._load_credentials()

    def _load_credentials(self) -> Optional[Credentials]:
        """Load credentials from file if they exist"""
        if self.token_file.exists():
            with open(self.token_file, "rb") as token:
                return pickle.load(token)
        return None

    def _save_credentials(self, credentials: Credentials):
        """Save credentials to file"""
        with open(self.token_file, "wb") as token:
            pickle.dump(credentials, token)

    def get_credentials(self) -> Optional[Credentials]:
        """Get current credentials, refreshing if necessary"""
        if not self.credentials:
            return None

        if self.credentials.expired:
            self.credentials.refresh(GoogleRequest())
            self._save_credentials(self.credentials)

        return self.credentials

    def authenticate(self):
        """Run OAuth flow to get credentials"""
        flow = InstalledAppFlow.from_client_secrets_file(
            self.secrets_file,
            scopes=[
                "https://www.googleapis.com/auth/webmasters.readonly",
                "https://www.googleapis.com/auth/webmasters",
            ],
        )
        self.credentials = flow.run_local_server(port=0)
        self._save_credentials(self.credentials)


In [3]:
# | hide
auth = GSCAuth(secrets_file="client_secrets.json")
auth.authenticate()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=379261216946-18ru6cgt2sha1grk8vr0tu81kv1p0jir.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A38579%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters&state=i3D1VAVR9KfJSGIyxiU80kCDYElzRE&access_type=offline
Opening in existing browser session.


In [5]:
# | export
def get_date_range(
    range_type: str = "today",
    days: Optional[int] = None,
    months: Optional[int] = None,
    start_date: Optional[str] = None,
    end_date: Optional[str] = None,
) -> Tuple[str, str]:
    """Generate date range for GSC queries (accounts for 3-day delay)"""
    today = datetime.now()
    latest_available = today - timedelta(days=3)

    if range_type == "today":
        date = latest_available.strftime("%Y-%m-%d")
        return date, date

    if range_type == "last_days" and days:
        end = latest_available
        start = end - timedelta(days=days)
        return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

    if range_type == "last_months" and months:
        end = latest_available
        start = end - timedelta(days=30 * months)
        return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

    if range_type == "custom" and start_date and end_date:
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        if end.date() > latest_available.date():
            end = latest_available
        return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

    raise ValueError("Invalid date range parameters")


In [7]:
# | export
def fetch_gsc_data(
    auth: GSCAuth,
    site_url: str,
    start_date: str,
    end_date: str,
    dimensions: List[str] = None,
    row_limit: int = 25000,
) -> List[Dict]:
    """Fetch data from Google Search Console"""
    if dimensions is None:
        dimensions = ["query", "page", "country", "device"]

    service = build("searchconsole", "v1", credentials=auth.get_credentials())

    request = {
        "startDate": start_date,
        "endDate": end_date,
        "dimensions": dimensions,
        "rowLimit": row_limit,
    }

    response = service.searchanalytics().query(siteUrl=site_url, body=request).execute()

    return response.get("rows", [])


In [8]:
# | export
def get_verified_sites(auth: GSCAuth) -> List[Dict]:
    """Get list of verified sites"""
    service = build("searchconsole", "v1", credentials=auth.get_credentials())
    sites_list = service.sites().list().execute()

    return [
        {"site_url": site["siteUrl"], "permission_level": site["permissionLevel"]}
        for site in sites_list.get("siteEntry", [])
        if site["permissionLevel"] != "siteUnverifiedUser"
    ]


In [9]:
# | hide

sites = get_verified_sites(auth)
sites


[{'site_url': 'sc-domain:azlzone.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:alainclean.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:smaagarden.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:tanorfix.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:awazly.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:kamcalorie.com', 'permission_level': 'siteFullUser'},
 {'site_url': 'sc-domain:leakdetectionkw.com',
  'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:protienai.com', 'permission_level': 'siteFullUser'},
 {'site_url': 'sc-domain:emdadelgaz.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:gpuvec.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:kareemai.com', 'permission_level': 'siteOwner'},
 {'site_url': 'sc-domain:shelid.com', 'permission_level': 'siteOwner'}]

In [12]:
# | hide
from pprint import pprint

start, end = get_date_range("last_days", 1)
data = fetch_gsc_data(auth, "sc-domain:kareemai.com", start, end)
pprint(data)


[{'clicks': 1,
  'ctr': 0.5,
  'impressions': 2,
  'keys': ['huawei 7i review',
           'https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%207i.html',
           'deu',
           'MOBILE'],
  'position': 3.5},
 {'clicks': 1,
  'ctr': 1,
  'impressions': 1,
  'keys': ['huawei free buds 7i',
           'https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%207i.html',
           'ind',
           'MOBILE'],
  'position': 4},
 {'clicks': 1,
  'ctr': 0.25,
  'impressions': 4,
  'keys': ['huawei freebuds 5i review',
           'https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%205i.html',
           'pol',
           'MOBILE'],
  'position': 1.25},
 {'clicks': 1,
  'ctr': 1,
  'impressions': 1,
  'keys': ['huawei freebuds 5i vs 7i',
           'https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%207i.html',
           'deu',
           'DESKTOP'],
  'position': 1},
 {'clicks': 1,
  'ctr': 1,
  'impressions': 1,
  'keys': ['hu